In [ ]:
__author__ = "Angus Liao 2022/12/27"

In [1]:
import pandas as pd

# 讀入資料

In [22]:
#用pandas的read_csv方法讀入csv檔，參數為檔案名稱
Electric = pd.read_csv("data/總表CSV.csv")
Electric

,Date,Time,Price,Theta,Load(kW)
0,2020/1/1,00:00,1.5,0.0,400
1,2020/1/1,00:15,1.5,0.0,400
2,2020/1/1,00:30,1.5,0.0,379
3,2020/1/1,00:45,1.5,0.0,384
4,2020/1/1,01:00,1.5,0.0,379
...,...,...,...,...,...
35132,2020/12/31,23:00,1.5,0.0,444
35133,2020/12/31,23:15,1.5,0.0,432
35134,2020/12/31,23:30,1.5,0.0,424
35135,2020/12/31,23:45,1.5,0.0,379


In [12]:
Load_List = Electric["Load(kW)"].tolist()  #將全時段負載量儲存成陣列
Time_List = Electric["Time"].tolist()  #將全時段時間儲存成陣列

#用於判斷是否為整點時間的陣列
Hour_List = ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", "09:00", "10:00", "11:00"
            ,"12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"]
        

# 取整點資料，負載量由15分、30分、45分與整點資料相加

In [27]:
#將2020/1/1 00:00負載量移除
Load_List.remove(Load_List[0])

# 把非整點時間Drop掉，每天從1點開始算

In [23]:
#len(Electric["Time"]) => Electric資料框Time那一行的資料長度
for i in range(len(Electric["Time"])):
    count = 0  #用於判斷是否不等於24個整點資料
    for j in range(0, 24):
        if Time_List[i] != Hour_List[j]:
            count += 1  #不符合一個整點陣列值就加1
            if count == 24:  #24個都不符合，就丟掉這筆資料
                Electric = pd.DataFrame(Electric, columns = Electric.columns[0:]).drop(i, axis = 0)
Electric = pd.DataFrame(Electric, columns = Electric.columns[0:]).drop(0, axis = 0)  #去掉總資料的第一筆資料2020/1/1 00:00
Electric

,Date,Time,Price,Theta,Load(kW)
4,2020/1/1,01:00,1.50,0.0,379
8,2020/1/1,02:00,1.50,0.0,367
12,2020/1/1,03:00,1.50,0.0,367
16,2020/1/1,04:00,1.50,0.0,367
20,2020/1/1,05:00,1.50,0.0,374
...,...,...,...,...,...
35120,2020/12/31,20:00,3.44,0.0,775
35124,2020/12/31,21:00,3.44,0.0,703
35128,2020/12/31,22:00,3.44,0.0,537
35132,2020/12/31,23:00,1.50,0.0,444


# 重新計算負載量

In [24]:
Electric = pd.DataFrame(Electric, columns = Electric.columns[0:]).drop("Load(kW)", axis = 1)  #將負載量那行資料去除
Electric

,Date,Time,Price,Theta
4,2020/1/1,01:00,1.50,0.0
8,2020/1/1,02:00,1.50,0.0
12,2020/1/1,03:00,1.50,0.0
16,2020/1/1,04:00,1.50,0.0
20,2020/1/1,05:00,1.50,0.0
...,...,...,...,...
35120,2020/12/31,20:00,3.44,0.0
35124,2020/12/31,21:00,3.44,0.0
35128,2020/12/31,22:00,3.44,0.0
35132,2020/12/31,23:00,1.50,0.0


# 計算每小時的負載量

In [28]:
sum = 0
total = []  #用於儲存各時段負載量的陣列

count = -1  #利用索引值呼叫原先負載量資料
for i in range(0, 35136, 4):  #每15分鐘為一個小時的資料，索引值一次跳4
    sum = 0
    for j in range(0, 4, 1):
        count += 1
        #print(i, j, count)
        sum += Load_List[count]  #累加負載量
    total.append(sum)  #將各時段計算結果存入陣列


total[0], total[1], total[2], total[3]

(1542, 1504, 1487, 1510)

In [29]:
Load_df = pd.DataFrame(total, columns = ["Load(kW)"])  #將全時段負載量陣列存成資料框
Load_df

,Load(kW)
0,1542
1,1504
2,1487
3,1510
4,1506
...,...
8779,3174
8780,2812
8781,2350
8782,1859


In [30]:
Electric.reset_index(inplace = True)  #更新資料框索引值
Electric

,index,Date,Time,Price,Theta
0,4,2020/1/1,01:00,1.50,0.0
1,8,2020/1/1,02:00,1.50,0.0
2,12,2020/1/1,03:00,1.50,0.0
3,16,2020/1/1,04:00,1.50,0.0
4,20,2020/1/1,05:00,1.50,0.0
...,...,...,...,...,...
8779,35120,2020/12/31,20:00,3.44,0.0
8780,35124,2020/12/31,21:00,3.44,0.0
8781,35128,2020/12/31,22:00,3.44,0.0
8782,35132,2020/12/31,23:00,1.50,0.0


In [31]:
Electric = Electric.drop(columns = ["index"])  #把舊的索引值移除
Electric

,Date,Time,Price,Theta
0,2020/1/1,01:00,1.50,0.0
1,2020/1/1,02:00,1.50,0.0
2,2020/1/1,03:00,1.50,0.0
3,2020/1/1,04:00,1.50,0.0
4,2020/1/1,05:00,1.50,0.0
...,...,...,...,...
8779,2020/12/31,20:00,3.44,0.0
8780,2020/12/31,21:00,3.44,0.0
8781,2020/12/31,22:00,3.44,0.0
8782,2020/12/31,23:00,1.50,0.0


In [32]:
Electric = pd.concat([Electric, Load_df], axis = 1)  #將資料框與負載量資料框結合
Electric

,Date,Time,Price,Theta,Load(kW)
0,2020/1/1,01:00,1.50,0.0,1542
1,2020/1/1,02:00,1.50,0.0,1504
2,2020/1/1,03:00,1.50,0.0,1487
3,2020/1/1,04:00,1.50,0.0,1510
4,2020/1/1,05:00,1.50,0.0,1506
...,...,...,...,...,...
8779,2020/12/31,20:00,3.44,0.0,3174
8780,2020/12/31,21:00,3.44,0.0,2812
8781,2020/12/31,22:00,3.44,0.0,2350
8782,2020/12/31,23:00,1.50,0.0,1859


In [33]:
Electric.to_csv('data/整點資料CSV.csv', index = False)  #存入資料夾